In [ ]:
from itertools import cycle
from typing import Optional, Union

import os
from datetime import datetime, timedelta
from time import sleep
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from glob import glob
import requests_cache
import numpy as np

In [ ]:
data_dir = '../database/'
df = pd.read_parquet(data_dir + 'latest.parquet')
df_smard = pd.read_parquet(data_dir + 'upd_smard_energy.parquet')
df_om = pd.read_parquet(data_dir + 'upd_openweather.parquet')
df_es = pd.read_parquet(data_dir + 'upd_epexspot.parquet')

In [ ]:
df.columns

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_multivariate_timeseries(df):
    # Create a subplot with 3 rows
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

    # Plot 'DA_auction_price' on the first subplot
    fig.add_trace(go.Scatter(x=df.index, y=df['DA_auction_price'], name='DA Auction Price'),
                  row=1, col=1)

    # Plot 'total_gen', 'total_grid_load', 'residual_load_forecast' on the second subplot
    fig.add_trace(go.Scatter(x=df.index, y=df['total_gen'], name='Total Generation'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['total_grid_load'], name='Total Grid Load'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['residual_load_forecast'], name='Residual Load Forecast'), row=2, col=1)

    # Plot 'temperature_2m_hsee' and 'temperature_2m_solw' on the third subplot
    fig.add_trace(go.Scatter(x=df.index, y=df['temperature_2m_hsee'], name='Temperature 2m HSEE'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['temperature_2m_solw'], name='Temperature 2m SOLW'), row=3, col=1)

    # Add a vertical line for today across all subplots
    # today_line = dict(type='line', x0=pd.Timestamp.today(), y0=0, x1=pd.Timestamp.today(),
    #                   y1=1, xref='x', yref='paper', line=dict(color='black', width=2, dash='dash'))
    # fig.add_shape(today_line, row='all', col=1)
    fig.add_vline(x=pd.Timestamp.today(), line_width=1, line_dash="dash", line_color="gray")

    # Update layout
    fig.update_layout(height=900, title='Multivariate Time Series Analysis',
                      xaxis_title='DateTime', yaxis_title='Values')

    # Show the figure
    fig.show()
plot_multivariate_timeseries(df=df)